In [72]:
import pandas as pd
import numpy as np

In [73]:
def repl(string):
    string1 = string.replace(']','').replace('[', '').replace('\'','')
    return string1

In [106]:
def one_decimal(rating):
    return round(rating, 1)

def categorize_ratings(df, col):
    df[col] = df[col].apply(one_decimal)
    return df
def lower(text):
    return text.lower()

def lower_cols(df, cols):
    for col in cols:
        df[col] = df[col].apply(lower)
    return df

def remove_spaces(text):
    text = text.replace(' ', '', 10)
    return text

def join_names(df, col):
    df[col] = df[col].apply(remove_spaces)
    return df



In [107]:
remove_spaces('stephen hawking')

'stephenhawking'

In [108]:
books_df = pd.read_csv('../data/small_dataset/gr_books_df_0.csv')
books_df['author_names'] = books_df['author_names'].apply(repl)
books_df = categorize_ratings(books_df,'avg_rating')
books_df = books_df[['isbn', 'book_title', 'avg_rating', 'author_names', 'book_description']]
books_df = lower_cols(books_df, ['book_title', 'author_names', 'book_description'])
books_df = join_names(books_df, 'author_names')

In [112]:
books_df['content'] = books_df['book_title'] + ' ' + books_df['author_names'] + ' ' + books_df['book_description']

In [113]:
books_df['content']

0     a brief history of time stephenhawking in the ...
1     a short history of nearly everything billbryso...
2     the selfish gene richarddawkins the selfish ge...
3     cosmos carlsagan cosmos has 13 heavily illustr...
4     the man who mistook his wife for a hat and oth...
                            ...                        
80    superintelligence: paths, dangers, strategies ...
81    the character of physical law richardp.feynman...
82    other minds: the octopus, the sea, and the dee...
83    the tell-tale brain: a neuroscientist's quest ...
84    the big picture: on the origins of life, meani...
Name: content, Length: 85, dtype: object

In [49]:
corpus = np.array(books_df['book_title'])

In [53]:
title = books_df['book_title'][1]

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

tfidf.fit(corpus)
document_tfidf_matrix = tfidf.transform(corpus)

print(tfidf.get_stop_words())
vocab = sorted(tfidf.vocabulary_)

None


In [46]:
avg_ratings = list(books_df['avg_rating'].unique())
book_titles = list(books_df['book_title'].unique())
author_names = list(books_df['book_title'].unique())
cat_features = avg_ratings + book_titles + author_names